In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.widgets import TextBox, Button
import sys
import pandas as pd
import os
import hfda

In [ ]:
from random import uniform
from scipy.fftpack import rfft,irfft,fft, ifft

def Random_phase_shuffling(signal):
    fft_signal = rfft(signal)
    phase_fs = np.arctan2(fft_signal[2::2],fft_signal[1:-1:2])
    mag = np.sqrt((fft_signal[1:-1:2])**2+(fft_signal[2::2])**2)
    ##phase shuffler:
    rng_phase = phase_fs.copy()
    np.random.shuffle(rng_phase)
    ##new signal : 
    fsrp = mag[:, np.newaxis] * np.c_[np.cos(rng_phase), np.sin(rng_phase)]
    fsrp = np.r_[fft_signal[0], fsrp.ravel(), fft_signal[-1]]
    tsr = irfft(fsrp)
    return tsr

In [ ]:
###plot of one attractor:

path = "/workspaces/maitrise/data/attractors"
df = pd.read_csv(path + "/lorenz__0.csv")
df_n = df.to_numpy()
print(df_n.shape)
xyzs = df_n[:,1:4]
t = df_n[:,0]
#print(df.to_string())



In [ ]:
##Plot attractor:
def Plot_attractors(xyzs_attractor):
    ax = plt.figure().add_subplot(projection='3d')

    ax.plot(*xyzs_attractor.T, lw=0.5)
    ax.set_xlabel("X Axis")
    ax.set_ylabel("Y Axis")
    ax.set_zlabel("Z Axis")
    ax.set_title("Lorenz Attractor")

    plt.show()

In [ ]:
##Plot attractor:
Plot_attractors(xyzs)

In [ ]:
##Adding noise yith specific SNR : 

def add_observational_noise(sig,SNR):
    fft_signal = rfft(sig)
    Power_sig = (fft_signal[1:-1:2])**2+(fft_signal[2::2])**2
    sd_noise = np.sqrt(Power_sig/SNR)
    noise = np.random.normal(0,sd_noise,len(sig))
    sig_noisy = sig+noise
    return sig_noisy




In [ ]:
##I1 and I2 thingy

def Mean_taux(signal,taux,hprime,h=0):
    if int(taux+h+hprime)<len(signal):
        return np.mean((signal[int(h+taux):int(taux+hprime+h)]))
    #elif len(signal)-h-hprime<taux:
    #    return np.mean((signal[int(h+taux):-1]))
    else: 
        return 0

def Variance_taux(signal,taux,hprime,h=0):
    if int(taux+h+hprime)<len(signal):
        return np.var((signal[int(h+taux):int(taux+hprime+h)]))
    #elif len(signal)-h-hprime<taux:
    #    return np.var((signal[int(h+taux):-1]))
    else: 
        return 0

def I1(c,signal,fs,tab,h,hprime,t0=0):
    if (t0>c or t0<0):
        return "error : t0 is outside the range you want to calculate"
    elif (c>len(signal) or c<0):
        return "error : c is outside the range of your signal"
    else: 
        #I1c = 0
        #for i in range(t0,int(c*fs)): 
        #    a =(1/(h*len(signal)))*np.abs(Mean_taux(signal,i,hprime*len(signal),h*len(signal))-Mean_taux(signal,i,hprime*len(signal)))
        #    I1c = I1c+a
        if len(tab) ==0:
            I1c = (1/(h*len(signal)))*np.abs(Mean_taux(signal,t0*fs,hprime*len(signal),h*len(signal))-Mean_taux(signal,t0*fs,hprime*len(signal)))
        else : 
            I1c = tab[-1]
            I1c += (1/(h*len(signal)))*np.abs(Mean_taux(signal,c*fs,hprime*len(signal),h*len(signal))-Mean_taux(signal,c*fs,hprime*len(signal)))

    return I1c

def I2(c,signal,fs,tab,h,hprime,t0=0):
    if (t0>c or t0<0):
        return "error : t0 is outside the range you want to calculate"
    elif (c>len(signal) or c<0):
        return "error : c is outside the range of your signal"
    else: 
        #I2c = 0
        #for i in range(t0,int(c*fs)): 
        #    a =  (1/(h*len(signal)))*np.abs(Variance_taux(signal,i,hprime*len(signal),h*len(signal))-Variance_taux(signal,i,hprime*len(signal)))
        #    I2c = I2c+a
        if len(tab) ==0:
            I2c = (1/(h*len(signal)))*np.abs(Variance_taux(signal,t0*fs,hprime*len(signal),h*len(signal))-Variance_taux(signal,t0*fs,hprime*len(signal)))
        else : 
            I2c = tab[-1]
            I2c += (1/(h*len(signal)))*np.abs(Variance_taux(signal,c*fs,hprime*len(signal),h*len(signal))-Variance_taux(signal,c*fs,hprime*len(signal)))

    return I2c

def discrepancies_mean_curve(signal,fs,h,hprime,t0=0):
    c =  np.linspace(t0,int((len(signal)/fs)),len(signal))
    I1_val = np.array([])
    I2_val = np.array([])
    for j in c:
            I1_val = np.append(I1_val,I1(j,signal,fs,I1_val,h,hprime,t0))
            I2_val = np.append(I2_val,I2(j,signal,fs,I2_val,h,hprime,t0))
    return I1_val,I2_val,c


    

In [ ]:
##Plotting I1 and I2
fs = 1/(t[1]-t[0])
dico_xyzs = {}
name = ["x","y","z"]
dico_xyzs[name[0]] = xyzs[:,0]
dico_xyzs[name[1]] = xyzs[:,1]
dico_xyzs[name[2]] = xyzs[:,2]


###Plot of signal first : 
rt = t*fs
for i in name:
    plt.plot(t,dico_xyzs[i],label = i)
plt.xlabel("time")
plt.ylabel("function value")
plt.legend(loc = "best", bbox_to_anchor=(1, 1))
plt.grid()
plt.plot()


for i in name:
    I1_c,I2_c,c = discrepancies_mean_curve(dico_xyzs[i],fs,0.001,0.1)
    fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(20,10))
    ax[0].set_title(i)
    ax[0].plot(c,I1_c)
    ax[0].set_xlabel("signal legnth [s]")
    ax[0].set_ylabel("I1 value")
    ax[0].grid()
    ax[1].set_title(i)
    ax[1].plot(c,I2_c)
    ax[1].set_xlabel("signal legnth [s]")
    ax[1].set_ylabel("I2 value")
    ax[1].grid()
    

In [ ]:
##TSD ofr observational noise
segment_length = 1.5
D_lead = {}
fs = 1/(t[1]-t[0])
dico_xyzs = {}
name = ["x","y","z"]
dico_xyzs[name[0]] = xyzs[:,0]
dico_xyzs[name[1]] = xyzs[:,1]
dico_xyzs[name[2]] = xyzs[:,2]
for i in name:
    w=1
    Ds = np.array([])
    sig = dico_xyzs[i]
    while (w*segment_length*fs)<=len(sig):
        sig_c  = sig[int((w-1)*segment_length*fs):int((w)*segment_length*fs)]
        Ds = np.append(Ds,hfda.measure(sig_c,50))
        w+=1
    D_lead[i] = Ds
print(D_lead)
w_length = [w*segment_length for w in range(0,int((len(sig)/fs)*(1/segment_length)))]


for i in name:
    plt.plot(w_length,D_lead[i],label = i)
    plt.xlabel(" Time interval")
    plt.ylabel("TSD value")
plt.legend(loc = "best", bbox_to_anchor=(1, 1))
plt.grid()
plt.show()



In [ ]:
###shuffling time!!!!!
def The_Big_shufflers(xyzs_clean):
    xyzs_shuffled=np.empty((df_n.shape[0],3))
    x = xyzs[:,0]
    y = xyzs[:,1]
    z = xyzs[:,2]
    x_noisy = Random_phase_shuffling(x)
    y_noisy = Random_phase_shuffling(y)
    z_noisy = Random_phase_shuffling(z)
    xyzs_shuffled[:,0] = x_noisy
    xyzs_shuffled[:,1] = y_noisy
    xyzs_shuffled[:,2] = z_noisy
    return xyzs_shuffled

    


In [ ]:
##plot shuffled attractor
xyzs_n = The_Big_shufflers(xyzs)
Plot_attractors(xyzs_n)